In [1]:
import copy
from collections import Counter

In [2]:
def read_instructions(fp):
    return [
        line.split(':')[0]
        for line in fp
        if ':' in line
    ]

In [3]:
target_name = 'stream_copy'
target = f'{target_name}/{target_name}'

lw_file_fp = open(f'{target}.lw', 'r')
sw_file_fp = open(f'{target}.sw', 'r')
trace_fp = open(f'{target}.txt', 'r')

traces = list(map(lambda x: x.split(','), trace_fp.read().split('\n')))
sw_list = read_instructions(sw_file_fp)
lw_list = read_instructions(lw_file_fp)
sw_set = set(sw_list)
lw_set = set(lw_list)

In [4]:
def translate(vaddr_str):
    vaddr = int(vaddr_str, base=16)
    if 0x80000000 <= vaddr <= 0x9FFFFFFF:
        addr = vaddr - 0x80000000
    elif 0xA0000000 <= vaddr <= 0xBFFFFFFF:
        addr = vaddr - 0xA0000000
    else:
        raise ValueError(vaddr_str)
    return '{:08x}'.format(addr)

In [5]:
x1faf_read_count = 0
x1faf_write_count = 0
x1fc_read_count = 0
x1fc_write_count = 0
instr_count = 0
sw_count = 0
lw_count = 0

last_vaddr = None
for rec in traces:
    if len(rec) == 1:
        continue
    elif len(rec) == 2:
        addr, channel = rec
        if addr.startswith('1faf'):
            if channel == 'aw':
                x1faf_write_count += 1
            else:
                x1faf_read_count += 1
        elif addr.startswith('1fc'):
            if channel == 'aw':
                x1fc_write_count += 1
            else:
                x1fc_read_count += 1
        else:
            raise RuntimeError(f'AXI: {addr}')
    else:
        vaddr, write_en, reg_num, data = rec
        if vaddr == last_vaddr:
            continue
        last_vaddr = vaddr
        instr_count += 1

        if vaddr in sw_set:
            sw_count += 1
        elif vaddr in lw_set:
            lw_count += 1

local_map = copy.copy(locals())
for key, value in local_map.items():
    if key.endswith('count'):
        print(f'{key}: {value}')
del local_map

x1faf_read_count: 7
x1faf_write_count: 155
x1fc_read_count: 18197
x1fc_write_count: 1202
instr_count: 14867
sw_count: 1199
lw_count: 3191


In [6]:
axi_count = x1faf_read_count + x1faf_write_count + x1fc_read_count + x1fc_write_count
print(f'axi_count = {axi_count}')

axi_count = 19561


In [7]:
cpu_count = instr_count + sw_count + lw_count
print(f'cpu_count = {cpu_count}')

cpu_count = 19257


In [8]:
diff = axi_count - cpu_count
print(f'diff = {diff}')

diff = 304
